In [1]:
import os
import sys

In [2]:
# Root directory of the project
ROOT_DIR = os.path.abspath(".")

In [3]:
sys.path.append('/home/israel/repos/Mask_RCNN/samples')  

In [4]:
from testfile import predict

ModuleNotFoundError: No module named 'mrcnn'

In [ ]:
model = predict()

In [ ]:
model.inference()

# Detectron2

In [ ]:
import os
import sys
sys.executable

In [ ]:
sys.path.append('/home/israel/repos/detectron2/demo/')  

In [ ]:
from predictor import VisualizationDemo

In [ ]:
model = VisualizationDemo(cfg)

In [ ]:
predictions, vis_output = model.run_on_image(im)

In [ ]:
predictions['instances'].pred_masks

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

In [ ]:
import numpy as np
import cv2
import random

In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
im = cv2.imread("/home/israel/repos/labelme/examples/test/2011_000025.jpg")

cfg = get_cfg()
cfg.merge_from_file("/home/israel/repos/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # set threshold for this model

In [ ]:
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.MODEL.DEVICE='cpu'
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes

In [ ]:
masks = outputs['instances'].pred_masks.to('cpu').numpy()
labels = outputs['instances'].pred_classes.to('cpu').numpy()

In [ ]:
shapes = []
for i, label in enumerate(labels):
    Mask(mask[0]).polygons()
    polygons = Mask(mask[i]).polygons()
    points = polygons.points[0][::5]
    points = list(map(tuple, points.astype('float')))
    instance = {'label': class_names[label],
                'points': points,
                'group_id': None,
                'shape_type': 'polygon',
                'flags': {}}
    shapes.append(instance)

In [ ]:
shapes

In [ ]:
plt.imshow(mask[0])
plt.show()

In [ ]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imwrite('output.png',v.get_image()[:, :, ::-1])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from imantics import Polygons, Mask
import cv2

In [ ]:
# This can be any array
array = mask[0]

polygons = Mask(array).polygons()

pol = cv2.polylines(im.copy(), [polygons.points[0][::2]], True, (0,255,0), 2)
plt.figure(figsize=(30,30))
plt.imshow(pol)
plt.show()

In [ ]:
from predict import predict

In [ ]:
model = predict()

In [ ]:
shape = model.inference('test')

In [ ]:
(list(map(tuple, shape.astype('float')))[0][0])

In [ ]:
former = [(152.53456221198158, 70.35483870967742), (108.7557603686636, 176.80645161290323), (270.5069124423963, 154.68663594470047)]

In [ ]:
type(former[0][0])

# TorchVision

In [ ]:
import sys
sys.executable

In [ ]:
from PIL import Image
# import imantics
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as T

In [ ]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
def get_prediction(img_path, threshold):
    img = Image.open(img_path)
    transform = T.Compose([T.ToTensor()])
    img = transform(img)
    pred = model([img])
    pred_score = list(pred[0]['scores'].detach().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
    masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy()
    pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
    masks = masks[:pred_t+1]
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    return masks, pred_boxes, pred_class

In [ ]:
path = "/home/israel/repos/labelme/examples/test/2011_000025.jpg"
masks, pred_boxes, pred_class = get_prediction(path, 0.5)

In [ ]:
plt.imshow(masks[0])
plt.show()

In [ ]:
import os
import re
import datetime
import numpy as np
from itertools import groupby
from skimage import measure
from PIL import Image
from pycocotools import mask

convert = lambda text: int(text) if text.isdigit() else text.lower()
natrual_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]

In [ ]:
def resize_binary_mask(array, new_size):
    image = Image.fromarray(array.astype(np.uint8)*255)
    image = image.resize(new_size)
    return np.asarray(image).astype(np.bool_)

def close_contour(contour):
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    return contour

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
                counts.append(0)
        counts.append(len(list(elements)))

    return rle

def binary_mask_to_polygon(binary_mask, tolerance=0):
    """Converts a binary mask to COCO polygon representation
    Args:
        binary_mask: a 2D binary numpy array where '1's represent the object
        tolerance: Maximum distance from original points of polygon to approximated
            polygonal chain. If tolerance is 0, the original coordinate array is returned.
    """
    polygons = []
    # pad mask to close contours of shapes which start and end at an edge
    padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
    contours = measure.find_contours(padded_binary_mask, 0.5)
    contours = np.subtract(contours, 1)
    for contour in contours:
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, tolerance)
        if len(contour) < 3:
            continue
        contour = np.flip(contour, axis=1)
        segmentation = contour.ravel().tolist()
        seg = segmentation
        # after padding and subtracting 1 we may get -0.5 points in our segmentation 
        segmentation = [0 if i < 0 else i for i in segmentation]
        polygons.append(segmentation)

    return polygons, seg, contour

In [ ]:
poly, seg, con = binary_mask_to_polygon(masks[0],0)

In [ ]:
points = con[::5]
points = list(map(tuple, points.astype('float')))

In [ ]:
points

In [ ]:
import mask2poly

In [ ]:
from mask2poly import binary_mask_to_polygon

In [ ]:
binary_mask_to_polygon(masks[0],0)[2]

In [5]:
from predict_torch import predict

In [6]:
model = predict()

In [7]:
pred = model.inference("/home/israel/repos/labelme/examples/test/2011_000025.jpg")

/home/israel/test_env/lib/python3.8/site-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


In [8]:
len(pred[0]['points'])

68